In [215]:
#imports

import requests
import pickle
import os
import tweepy
import sys

In [216]:
#This stuff supports the tinyurl block
from __future__ import with_statement                                                        
import contextlib
try:
    from urllib.parse import urlencode          
except ImportError:
    from urllib import urlencode 
try:
    from urllib.request import urlopen 
except ImportError:
    from urllib2 import urlopen 


In [217]:
# This will post a blog entry
def MakeABlogPost(Title, Body):

    # Save the thing we want to post to a file
    ToPost={"title":Title,"content":Body}
    f=open("./ToPost.pkl",'wb')
    pickle.dump(ToPost,f)
    f.close()

    # Run this python script that does the business
    stream = os.popen('python3 blogger.py --noauth_local_webserver')
    output = stream.read()
    
    #tidy up and json-ify output
    return output


# This will make a blog tweet
def MakeATweet(text):

    #Load twitter credential keys from file
    f=open("./TwitterCreds.pkl",'rb')
    TwitterCreds=pickle.load(f)
    f.close()
                             
    #Open the twitter API
    api = tweepy.Client(bearer_token    = TwitterCreds['bearer_token'],
                    access_token        = TwitterCreds['access_token'],
                    access_token_secret = TwitterCreds['access_token_secret'],
                    consumer_key        = TwitterCreds['consumer_key'],
                    consumer_secret     = TwitterCreds['consumer_secret_key'])

    # Create a tweet
    api.create_tweet(text=text)
    

# This will make a shortened url
def make_tiny(url):
    request_url = ('http://tinyurl.com/api-create.php?' + urlencode({'url':url}))   
    with contextlib.closing(urlopen(request_url)) as response:                      
        return response.read().decode('utf-8 ') 

    

In [220]:
#Put it all together and issue a horoscope

def IssueHoroscope(BlogTitle,BlogText,TweetText):
    if((len(TweetText) + 48)>275):
        print("Warning, your tweet is too many characters, I am not going to post it!")
        return -1

    output=MakeABlogPost(BlogTitle,BlogText)
    returnstuff=output[output.find("{"):].replace("'",'"').replace('""','"')[:-1]
    paresedoutput=json.loads(returnstuff)

    url= make_tiny(paresedoutput['url'])

    MakeATweet(TweetText+" Full horoscope at: " +str(url))
    return 0


In [221]:
# Example of how to use it!

BlogTitle="Neutrino observed in Taurus!"
BlogText="A new neutrino candidate was detected from Taurus! It means you will get rich!"
TweetText="A new neutrino candidate was detected from Taurus!"

IssueHoroscope(BlogTitle,BlogText,TweetText)

0